In [1]:
import pandas as pd

accomodations_clusters = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/clusters_central_location.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/time_matrix.csv')
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/accomodations_to_venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/datasets/bus_terminals.csv')
merged_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/merged_matrix.csv')

In [2]:
venues = venues.dropna()

In [3]:
import folium

# Create a map centered at the average latitude and longitude
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=10)

# Add markers for each cluster
for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m)


# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m)
    
# Add markers for each bus terminal
for _, row in bus_terminals.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='green')
    ).add_to(m)
# Display the map
m

In [ ]:
demand= accomodations_clusters['total_accommodates'].to_numpy()
demand

array([ 6969,  6395,  2801,  7026,  4909,  1545,  6342,  6399,  1171,
        4117,  3110,  3088,  5080,  4779,  2666,  9095,  7555,  4679,
        2111,  3376,  7538,  7068,  6853,  1576,  2974,  2110,  1319,
       10383,  1268,   273,  8063,  8239,  3833,  6408,  7028,  2057,
        4099,  2060,  2646,  2505,  6751,   934,  4119,  9229,   563,
        7296,  2670,  2156,  2263,  1619, 12812])

: 

In [ ]:
from ortools.sat.python import cp_model

# Set the number of bus stops to locate
p = 15

# bus_matrix rows: accommodations; columns: candidate bus terminals
accom = list(bus_matrix.index)
bus_stops = list(bus_matrix.columns[1:])

model = cp_model.CpModel()

# Decision variables:
x = {j: model.NewBoolVar(f"x_{j}") for j in bus_stops}
y = {}
for idx, i in enumerate(accom):
    for j in bus_stops:
        y[(idx, j)] = model.NewBoolVar(f"y_{idx}_{j}")

# Constraint: each accommodation must be assigned to exactly one bus stop.
for idx, i in enumerate(accom):
    model.Add(sum(y[(idx, j)] for j in bus_stops) == 1)

# Constraint: assignment only possible if bus stop is selected.
for idx, i in enumerate(accom):
    for j in bus_stops:
        model.Add(y[(idx, j)] <= x[j])

# Constraint: exactly p bus stops are selected.
model.Add(sum(x[j] for j in bus_stops) == p)

# Objective: minimize total (demand-weighted) distance.
# We convert bus_matrix.loc[i, j] to an integer cost.
objective_terms = []
for idx, i in enumerate(accom):
    for j in bus_stops:
        # Multiply demand and the scaled distance.
        cost = int(round(bus_matrix.loc[i, j]))
        objective_terms.append(demand[idx] * cost * y[(idx, j)])
model.Minimize(sum(objective_terms))

# Solve the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    selected = [j for j in bus_stops if solver.Value(x[j]) == 1]
    print("Selected bus stops:", selected)
else:
    print("No solution found.")

In [ ]:
#map only selected bus stops ['BD14', 'BD15', 'BT03', 'BL14', 'BT23', 'BT18', 'BL23', 'BL07', 'BT05']
# Add markers for each cluster
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m2 = folium.Map(location=map_center, zoom_start=10)

for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m2)
# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m2)

for _, row in bus_terminals[bus_terminals['id'].isin(selected)].iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='lightgreen', icon='ok-sign')
    ).add_to(m2)
# Display the map with selected bus stops
m2

In [ ]:
merged_matrix

,Unnamed: 0,A1,A10,A11,A12,A13,A14,A15,A16,A17,...,BT19,BT20,BT21,BT22,BT23,BT24,BT25,BT26,BT27,BT28
0,A1,0.0,3304.0,3496.0,1727.0,1956.0,2020.0,2410.0,1512.0,3273.0,...,1850.0,2149.0,1860.0,2759.0,2024.0,2357.0,2234.0,1594.0,1485.0,1745.0
1,A10,3263.0,0.0,4901.0,2745.0,2974.0,3033.0,2460.0,2917.0,2778.0,...,1696.0,1627.0,2491.0,2459.0,2890.0,2960.0,2575.0,2406.0,2420.0,2006.0
2,A11,3459.0,4958.0,0.0,3305.0,3534.0,3598.0,4079.0,2432.0,4852.0,...,3477.0,3757.0,3064.0,4313.0,3578.0,3911.0,3788.0,3326.0,3408.0,3477.0
3,A12,1753.0,2784.0,3117.0,0.0,593.0,1014.0,1617.0,1405.0,2390.0,...,1556.0,1785.0,2037.0,1875.0,1140.0,1473.0,1350.0,1783.0,1424.0,1630.0
4,A13,1909.0,2939.0,3272.0,576.0,0.0,489.0,1654.0,1561.0,2426.0,...,1711.0,1940.0,2193.0,1911.0,1176.0,1509.0,1386.0,1938.0,1579.0,1785.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,BT24,2357.0,2960.0,3911.0,1473.0,1509.0,1455.0,1027.0,2040.0,1565.0,...,1847.0,1958.0,2368.0,1327.0,980.0,0.0,934.0,1965.0,1990.0,1634.0
150,BT25,2234.0,2575.0,3788.0,1350.0,1386.0,1332.0,250.0,1918.0,1442.0,...,1126.0,1238.0,1647.0,840.0,857.0,952.0,0.0,1244.0,1479.0,914.0
151,BT26,1594.0,2406.0,3326.0,1783.0,1938.0,2233.0,1496.0,1365.0,2523.0,...,933.0,1044.0,1418.0,1854.0,2018.0,2087.0,1345.0,0.0,357.0,707.0
152,BT27,1485.0,2420.0,3408.0,1424.0,1579.0,1874.0,1556.0,1447.0,2584.0,...,1164.0,1311.0,1731.0,1914.0,1635.0,2047.0,1405.0,362.0,0.0,987.0


# VRP